In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U sentence-transformers
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cpu"


In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('keepitreal/vietnamese-sbert')
model = AutoModel.from_pretrained('keepitreal/vietnamese-sbert')

# # Path to your trained model's pytorch_model.bin file
# model_path = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection/huggingface/'

# # Load the model
# model = SentenceTransformer(model_path)

In [ ]:
# model.cuda()

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['Cô giáo đang ăn kem', 'Cô giáo đang ăn bánh kem']

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
encoded_input = encoded_input.to(device)

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 0.5585,  0.1187, -0.0377,  ..., -0.0916,  0.2738, -0.0168],
        [ 0.3344,  0.0230,  0.0007,  ..., -0.0423,  0.1422, -0.0766]])


In [ ]:
util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1])

tensor([[0.9533]])

In [ ]:
def Encode(claim, sents):
  encoded_claim = tokenizer(claim, padding=True, truncation=True, return_tensors='pt')
  encoded_sents = tokenizer(sents, padding=True, truncation=True, return_tensors='pt')
  encoded_claim = encoded_claim.to(device)
  encoded_sents = encoded_sents.to(device)

  with torch.no_grad():
    model_output_claim = model(**encoded_claim)
    model_output_sents = model(**encoded_sents)


  # Perform pooling. In this case, mean pooling.
  claim_embeddings = mean_pooling(model_output_claim, encoded_claim['attention_mask'])
  sents_embeddings = mean_pooling(model_output_sents, encoded_sents['attention_mask'])

  claim_embeddings = claim_embeddings.to(device)
  sents_embeddings = sents_embeddings.to(device)
  return claim_embeddings, sents_embeddings

In [ ]:
def sentence_similarity(claim, sents):
  claim_embeddings, sents_embeddings = Encode(claim, sents)
  result = {}
  for idx, sent in enumerate(sents_embeddings):
    cos_sim = util.pytorch_cos_sim(claim_embeddings, sent)
    result[idx] = cos_sim
  return result


In [ ]:
def get_result(result):
  # Sắp xếp dict theo giá trị
  sorted_dict = sorted(result.items(), key=lambda x: x[1], reverse=True)

  # Lấy khóa có giá trị lớn nhất
  key_with_max_value = sorted_dict[0][0]
  max_value = sorted_dict[0][1]
  return key_with_max_value, max_value.item()

In [ ]:
data_path = '/content/drive/MyDrive/pRoBERTa/data/converted_data_warmup_split.json'

In [ ]:
import json

# Đọc dữ liệu từ file JSON
with open(data_path) as file:
    data = json.load(file)

# Tạo một từ điển để lưu trữ dữ liệu đã chuyển đổi
load_data = {}

# Lặp qua mỗi ID trong data
for id, item in data.items():
    data = {}
    data["context"] = item['context']
    data["claim"] = item["claim"]
    if "public-test" not in data_path:
      data["evidence"] = item["evidence"]
    # Lưu trữ data vào từ điển theo ID tương ứng
    load_data[id] = data

In [ ]:
for id, item in tqdm(load_data.items()):
   result = sentence_similarity(item['claim'], item['context'])
   id_sent, conf = get_result(result)
   load_data[id]['evidence_predict'] = item['context'][id_sent]
   load_data[id]['idx_evidence'] = id_sent
   load_data[id]['percentages_rationale'] = conf


In [ ]:
load_data

# Lưu kết quả

In [ ]:
import os
import json

def save_result(PATH, result_dict):
    # Tạo thư mục nếu nó chưa tồn tại
    os.makedirs(PATH, exist_ok=True)
    # Lưu từ điển vào tệp JSON
    result_path = os.path.join(PATH, "result_warmup_rationale_VNsbert_1.json")

    with open(result_path, "w", encoding="utf-8") as file:
        json.dump(load_data, file, ensure_ascii=False)

In [ ]:
save_path = "/content/drive/MyDrive/pRoBERTa/source/result"
save_result(save_path, load_data)

In [ ]:
a = {"context": ["vận động viên chạy bộ người bulgaria krasse gueorguiev bắt đầu thử thách từ ngày 30/4", "anh sẽ sống 15 ngày trong một chiếc hộp có ba mặt bằng kính bên trong có giường và máy chạy bộ nhưng không có sách máy tính hay điện thoại", "gueorguiev được phép trò chuyện với mọi người 30 phút mỗi ngày", "tôi muốn thử thách bản thân gueorguiev nói", "tôi muốn cho mọi người thấy sự thay đổi tâm lý khi ai đó được đặt vào khuôn khổ", "thông qua thử thách gueorguiev muốn gây quỹ cho nhiều dự án có mục tiêu ngăn tình trạng nghiện ngập ở người dưới 18 tuổi bao gồm nghiện chất kích thích rượu thiết bị kỹ thuật số và đồ uống tăng lực", "gueorguiev là một diễn giả truyền cảm hứng và đại sứ từ thiện", "anh đã tham gia khoảng 30 giải siêu marathon trên thế giới trong đó có giải chạy 217 km ở thung lũng chết bang california mỹ"], "claim": "gueorguiev được phép trò chuyện với người khác khi thực hiện thử thách", "verdict": "SUPPORTED", "evidence": "gueorguiev được phép trò chuyện với mọi người 30 phút mỗi ngày", "domain": "thegioi"}

In [ ]:
a

{'context': ['vận động viên chạy bộ người bulgaria krasse gueorguiev bắt đầu thử thách từ ngày 30/4',
  'anh sẽ sống 15 ngày trong một chiếc hộp có ba mặt bằng kính bên trong có giường và máy chạy bộ nhưng không có sách máy tính hay điện thoại',
  'gueorguiev được phép trò chuyện với mọi người 30 phút mỗi ngày',
  'tôi muốn thử thách bản thân gueorguiev nói',
  'tôi muốn cho mọi người thấy sự thay đổi tâm lý khi ai đó được đặt vào khuôn khổ',
  'thông qua thử thách gueorguiev muốn gây quỹ cho nhiều dự án có mục tiêu ngăn tình trạng nghiện ngập ở người dưới 18 tuổi bao gồm nghiện chất kích thích rượu thiết bị kỹ thuật số và đồ uống tăng lực',
  'gueorguiev là một diễn giả truyền cảm hứng và đại sứ từ thiện',
  'anh đã tham gia khoảng 30 giải siêu marathon trên thế giới trong đó có giải chạy 217 km ở thung lũng chết bang california mỹ'],
 'claim': 'gueorguiev được phép trò chuyện với người khác khi thực hiện thử thách',
 'verdict': 'SUPPORTED',
 'evidence': 'gueorguiev được phép trò chuy

In [ ]:
result = sentence_similarity(a["claim"], a["context"])

In [ ]:
result

{0: tensor([[0.3719]]),
 1: tensor([[0.2257]]),
 2: tensor([[0.5892]]),
 3: tensor([[0.7487]]),
 4: tensor([[0.0879]]),
 5: tensor([[0.4817]]),
 6: tensor([[0.5159]]),
 7: tensor([[0.0364]])}

In [ ]:
id, val = get_result(result)

In [ ]:
a["context"][id]

'tôi muốn thử thách bản thân gueorguiev nói'